In [35]:
import sys
import csv

def read_from_csv(filename):
    csv.field_size_limit(sys.maxsize)
    with open(filename, 'r', newline='') as f:
        reader = csv.reader(f)
        next(reader)

        change_info_list = []
        for row in reader:
            change_info_list.append({
                "id": row[0],
                "gerrit_change_id": row[1],
                "project": row[2],
                "subject": row[3],
                "created": row[4],
                "updated": row[5],
                "status": row[6],
                "insertions": row[7],
                "deletions": row[8],
                "total_comment_count": row[9],
                "unresolved_comment_count": row[10],
                "messages": row[11],
                "comments": row[12],
                "added_methods": row[13],
                "removed_methods": row[14],
                "edited_methods": row[15],
                "diff_hunk": row[16]
            })

        return change_info_list
    
# Test the function
change_info_list = read_from_csv('dataset.csv')
len(change_info_list)

10663

In [2]:
# count all entries in change_info_list with status 'MERGED'
count = 0
merged_entries = []

for change_info in change_info_list:
    if change_info['status'] == 'MERGED':
        count += 1
        merged_entries.append(change_info)

print(count)

9765


In [3]:
# count all entries in change_info_list with status not 'MERGED'
count = 0
non_merged_entries = []

for change_info in change_info_list:
    if change_info['status'] != 'MERGED':
        count += 1
        non_merged_entries.append(change_info)

print(count)

898


In [4]:
# # select 898 random entries from merged_entries and create a balanced dataset with the non_merged_entries
import random

# Select 898 random entries from merged_entries
balanced_merged_entries = random.sample(merged_entries, 898)

# Combine with non_merged_entries
balanced_dataset = balanced_merged_entries + non_merged_entries

change_info_list = balanced_dataset

print(len(balanced_dataset))  # Should print 1796

1796


In [5]:
print(change_info_list[4]['added_methods'])

public IAtsInsertion getInsertion(ArtifactId artifact) {
public IAtsInsertionActivity getInsertionActivity(ArtifactId artifact) {
public void test() {
public void test() {
public void test() {
public void test() {
private void assertResult(Pair<? extends Object, Boolean> result, Object object, boolean inherited) {
public static WorkPackageUtility getUtil() {
public Country(Log logger, IAtsServices services, ArtifactId artifact) {


In [6]:
print(change_info_list[4]['added_methods'])

public IAtsInsertion getInsertion(ArtifactId artifact) {
public IAtsInsertionActivity getInsertionActivity(ArtifactId artifact) {
public void test() {
public void test() {
public void test() {
public void test() {
private void assertResult(Pair<? extends Object, Boolean> result, Object object, boolean inherited) {
public static WorkPackageUtility getUtil() {
public Country(Log logger, IAtsServices services, ArtifactId artifact) {


In [7]:
print(change_info_list[6]['comments'])

[{'filename': 'plugins/org.eclipse.osee.ats.api/src/org/eclipse/osee/ats/api/workflow/transition/IAtsTransitionManager.java', 'message': 'Missing @author', 'updated': '2014-03-03 16:24:50.000000000', 'commit_id': '8ca3977b3498e2a0b53344c0e9e02dc272770ae9', 'unresolved': False}, {'filename': 'plugins/org.eclipse.osee.ats.api/src/org/eclipse/osee/ats/api/workflow/transition/IAtsTransitionManager.java', 'message': 'Done', 'updated': '2014-03-04 03:26:02.000000000', 'commit_id': '8ca3977b3498e2a0b53344c0e9e02dc272770ae9', 'unresolved': False}, {'filename': 'plugins/org.eclipse.osee.ats.api/src/org/eclipse/osee/ats/api/workflow/transition/IAtsTransitionManager.java', 'message': 'Oops, you still missed the @Author annotation', 'updated': '2014-03-04 16:13:51.000000000', 'commit_id': '57c534847a1c73ea013ec749612f32c9e3a55cbd', 'unresolved': False}, {'filename': 'plugins/org.eclipse.osee.ats.api/src/org/eclipse/osee/ats/api/workflow/transition/IAtsTransitionManager.java', 'message': 'Oops, fix

In [8]:
print(change_info_list[4]['edited_methods'])

public IAtsCountry getCountry(ArtifactId artifact)


In [9]:
# for each row, convert the change_info_list[i]['messages'] and ['comments'] from a string to a list of dictionaries
for i in range(len(change_info_list)):
    change_info_list[i]['messages'] = eval(change_info_list[i]['messages'])
    change_info_list[i]['comments'] = eval(change_info_list[i]['comments'])

In [10]:
# create a new dataframe, with the same length of the change_info_list
import pandas as pd

df = pd.DataFrame(index=range(len(change_info_list)), columns=['message', 'status'])
# df = pd.DataFrame(index=range(len(balanced_dataset)), columns=['message', 'status'])

# MAX_LENGTH = 512
MAX_LENGTH = 4096

# edit the following columns: messages, comments, added_methods, removed_methods, edited_methods, diff_hunk
messages = []
for i in range(len(change_info_list)):
    messages_array = change_info_list[i]['messages']
    messages_list = ''
    for j in range(len(messages_array)):
        messages_list += str(j+1) + '. ' + messages_array[j]['message'] + '\n'

    # cap the maximal length of the messages_list to MAX_LENGTH characters (get the last MAX_LENGTH characters if the length is greater than MAX_LENGTH and start from the next line that starts with a number)
    if len(messages_list) > MAX_LENGTH:
        messages_list = messages_list[-MAX_LENGTH:]
        for k in range(MAX_LENGTH):
            if messages_list[k].isdigit():
                messages_list = messages_list[k:]
                break

    messages.append(messages_list)

comments = []
for i in range(len(change_info_list)):
    comments_array = change_info_list[i]['comments']
    comments_list = ''
    for j in range(len(comments_array)):
        comments_list += str(j+1) + '. '  + comments_array[j]['message'] + '\n'

    # cap the maximal length of the comments_list to MAX_LENGTH characters (get the last MAX_LENGTH characters if the length is greater than MAX_LENGTH and start from the next line that starts with a number)
    if len(comments_list) > MAX_LENGTH:
        comments_list = comments_list[-MAX_LENGTH:]
        for k in range(MAX_LENGTH):
            if comments_list[k].isdigit():
                comments_list = comments_list[k:]
                break

    comments.append(comments_list)

# make sure to limit every list to MAX_LENGTH characters
added_methods = []
removed_methods = []
edited_methods = []
diff_hunk = []
for i in range(len(change_info_list)):
    added_methods_list = change_info_list[i]['added_methods']
    removed_methods_list = change_info_list[i]['removed_methods']
    edited_methods_list = change_info_list[i]['edited_methods']
    diff_hunk_list = change_info_list[i]['diff_hunk']

    if len(added_methods_list) > MAX_LENGTH:
        added_methods_list = added_methods_list[:MAX_LENGTH]
    if len(removed_methods_list) > MAX_LENGTH:
        removed_methods_list = removed_methods_list[:MAX_LENGTH]
    if len(edited_methods_list) > MAX_LENGTH:
        edited_methods_list = edited_methods_list[:MAX_LENGTH]
    if len(diff_hunk_list) > MAX_LENGTH:
        diff_hunk_list = diff_hunk_list[:MAX_LENGTH]

    added_methods.append(added_methods_list)
    removed_methods.append(removed_methods_list)
    edited_methods.append(edited_methods_list)
    diff_hunk.append(diff_hunk_list)

# copy the colums listed below from change_info_list to df:
# project, subject, created, updated, insertions, deletions, total_comment_count, unresolved_comment_count
for i in range(len(change_info_list)):
    # text = "project: " + change_info_list[i]['project'] + "\nsubject: " + change_info_list[i]['subject'] + "\ncreated: " + change_info_list[i]['created'] + "\nupdated: " + change_info_list[i]['updated'] + "\ninsertions: " , change_info_list[i]['insertions'] + "\ndeletions: " + change_info_list[i]['deletions'] + "\ntotal comments: " + change_info_list[i]['total_comment_count'] + "\nunresolved comments: " + change_info_list[i]['unresolved_comment_count'] + "\nmessages: " + messages[i] + "\ncomments: " + comments[i] + "\nadded methods: " + added_methods[i] + "\nremoved methods: " + removed_methods[i] + "\nedited methods: " + edited_methods[i] + "\ndiff hunk:" + diff_hunk[i]
    # text = "project: " + change_info_list[i]['project'] + "\nsubject: " + change_info_list[i]['subject']
    text = "title:\n" + change_info_list[i]['subject'] + "\n\ndiff hunk:" + diff_hunk[i]
    df.loc[i] = [str(text), change_info_list[i]['status']]

print(df.head())

                                             message  status
0  title:\ntraceevent.ui: add object life span an...  MERGED
1  title:\ntmf: Add equals/hashCode to TmfConstan...  MERGED
2  title:\nBug 464639 - DsfSourceLookupParticipan...  MERGED
3  title:\n[Bug 340300] Fix parallel make when us...  MERGED
4  title:\nfeature[ats_ATS186279]: Performance Re...  MERGED


In [11]:
print(added_methods[4])
len(added_methods[4])


public IAtsInsertion getInsertion(ArtifactId artifact) {
public IAtsInsertionActivity getInsertionActivity(ArtifactId artifact) {
public void test() {
public void test() {
public void test() {
public void test() {
private void assertResult(Pair<? extends Object, Boolean> result, Object object, boolean inherited) {
public static WorkPackageUtility getUtil() {
public Country(Log logger, IAtsServices services, ArtifactId artifact) {


433

In [12]:
df['idx'] = df.index


# remove columns that we want to ignore for now
if 'project' in df.columns:
    df = df.drop(columns=['project'])

In [13]:
# preprocess the data for fine tuning
from datasets import ClassLabel, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

train, test = train_test_split(df, test_size=0.2, stratify=df['status']) # this splits the data and shuffles the entries
train, val = train_test_split(train, test_size=0.2, stratify=train['status'])

print(len(train))
# # balance the training set
# positive_samples = train[train['status'] == 'MERGED']
# negative_samples = train[train['status'] != 'MERGED']
# if len(positive_samples) > len(negative_samples):
#     positive_samples = resample(positive_samples, replace=False, n_samples=len(negative_samples), random_state=42)
# else:
#     negative_samples = resample(negative_samples, replace=False, n_samples=len(positive_samples), random_state=42)

# balanced_train = pd.concat([positive_samples, negative_samples], ignore_index=True)

# train_dataset = Dataset.from_pandas(balanced_train).shuffle(seed=42)

train_dataset = Dataset.from_pandas(train).shuffle(seed=42)
val_dataset = Dataset.from_pandas(val)
test_dataset = Dataset.from_pandas(test)

raw_datasets = DatasetDict({
    'train': train_dataset, 
    'validation': val_dataset, 
    'test': test_dataset
})

print(len(raw_datasets['train']))
# Normalize the 'status' column to lowercase
def recategorize_status(example):
    # Normalize the status to lowercase
    normalized_status = example['status'].lower()
    # Recategorize status
    if normalized_status == 'merged':
        example['status'] = 'merged'
    else:
        example['status'] = 'not_merged'
    return example

# Normalize and recategorize status in all splits
for split in raw_datasets.keys():
    raw_datasets[split] = raw_datasets[split].map(recategorize_status)
    
# create the ClassLabel feature
class_label_feature = ClassLabel(num_classes=2, names=['not_merged', 'merged'])

for split in raw_datasets.keys():
    raw_datasets[split] = raw_datasets[split].cast_column('status', class_label_feature)

# rename the 'status' column to 'label'
for split in raw_datasets.keys():
    raw_datasets[split] = raw_datasets[split].rename_column('status', 'label')

# remove the __index_level... columns if it exists (just if cell gets ran multiple times)
if '__index_level_0__' in raw_datasets['train'].column_names:
    raw_datasets['train'] = raw_datasets['train'].remove_columns('__index_level_0__')

if '__index_level_0__' in raw_datasets['test'].column_names:
    raw_datasets['test'] = raw_datasets['test'].remove_columns('__index_level_0__')
    
if '__index_level_0__' in raw_datasets['validation'].column_names:
    raw_datasets['validation'] = raw_datasets['validation'].remove_columns('__index_level_0__')

raw_datasets


1148
1148


Map:   0%|          | 0/1148 [00:00<?, ? examples/s]

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1148 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/288 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/360 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['message', 'label', 'idx'],
        num_rows: 1148
    })
    validation: Dataset({
        features: ['message', 'label', 'idx'],
        num_rows: 288
    })
    test: Dataset({
        features: ['message', 'label', 'idx'],
        num_rows: 360
    })
})

In [14]:
# print random entry from training set
print(raw_datasets['train'][5])
print(len(raw_datasets['train'][5]['message']))

{'message': 'title:\n[DRAFT 471900] Add classes for opening implementation of services\n\ndiff hunk:From 9f4b99efbf835234b1cb6b232d45bc419f2e3c5c Mon Sep 17 00:00:00 2001\nFrom: Benjamin Grouhan <benjamin.grouhan@obeo.fr>\nDate: Wed, 29 Jul 2015 15:35:19 +0200\nSubject: [PATCH] [DRAFT 471900] Add classes for opening implementation of services\n\nAdd an interface IImplementationProvider permitting to retrieve\nimplementations of services, and make the ServiceInterpreter implement\nthis interface. A method to retreieve the underlying Method object has\nbeen added in IMonomorphicService in order to do this.\n\nAlso add a class OpenImplementation, allowing to open the java files\ncorresponding to the implementations of a service from an interpreted\nexpression. For this new class, a dependency to org.eclipse.jdt.ui is\nnecessary, so this has been added in the manifest.mf of\norg.eclipse.sirius.editor.\n\nBug: 471900\nChange-Id: I7043750751540102f2c2281e774fbf19eafd6b2b\nSigned-off-by: Benj

In [15]:
# show a slice of the dataset
raw_datasets['train'][:30]

{'message': ["title:\nBug 515697 - Rename Resource - Out of sync should offer option to sync\n\ndiff hunk:From aee3004f2755266ba85b0b92ead213a6956c3954 Mon Sep 17 00:00:00 2001\nFrom: Jonas Hungershausen <jonas.hungershausen@vogella.com>\nDate: Thu, 30 Aug 2018 11:07:11 +0200\nSubject: [PATCH] Bug 515697 - Rename Resource - Out of sync should offer option to sync\n\nInstead of just checking if a file in the folder that is refactored is\nout of sync, we refresh the given resource to avoid the dialog that the\nresource is out of sync that doesn't provide any action for the user.\n\nChange-Id: I0928613154c847025b970f4f52af13267615aa9f\nSigned-off-by: Jonas Hungershausen <jonas.hungershausen@vogella.com>\n---\n\ndiff --git a/org.eclipse.ltk.core.refactoring/META-INF/MANIFEST.MF b/org.eclipse.ltk.core.refactoring/META-INF/MANIFEST.MF\nindex a5ba95e..766e5f3 100644\n--- a/org.eclipse.ltk.core.refactoring/META-INF/MANIFEST.MF\n+++ b/org.eclipse.ltk.core.refactoring/META-INF/MANIFEST.MF\n@@ -3

In [16]:
raw_datasets['test'][:30]

{'message': ['title:\nMore JUnit tests for the notification system (added tests: edit file, edit model element, edit Java method)\n\ndiff hunk:From 007ed97b0d1341f16d5ab69d0a4a61a04f24aa19 Mon Sep 17 00:00:00 2001\nFrom: Dusan Kalanj <kalanj@chalmers.se>\nDate: Fri, 02 Dec 2016 12:02:36 +0100\nSubject: [PATCH] More JUnit tests for the notification system (added tests: edit file, edit model element, edit Java method)\n\nChange-Id: Iac436b1a0c7e768c0ba052a4a916e2df58eb5168\nSigned-off-by: Dusan Kalanj <kalanj@chalmers.se>\n---\n\ndiff --git a/org.eclipse.capra.testsuite/src/org/eclipse/capra/testsuite/TestHelper.java b/org.eclipse.capra.testsuite/src/org/eclipse/capra/testsuite/TestHelper.java\nindex 88862f9..dcd3053 100644\n--- a/org.eclipse.capra.testsuite/src/org/eclipse/capra/testsuite/TestHelper.java\n+++ b/org.eclipse.capra.testsuite/src/org/eclipse/capra/testsuite/TestHelper.java\n@@ -68,8 +68,28 @@\n import org.eclipse.ui.IWorkbenchWindow;\n import org.eclipse.ui.PlatformUI;\n \n

In [17]:
raw_datasets['train'].features

{'message': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_merged', 'merged'], id=None),
 'idx': Value(dtype='int64', id=None)}

In [18]:
# import tensorflow as tf
# tf.keras.backend.clear_session()  # This clears the session


In [19]:
# count how many entries in the training set are 'merged' and 'not_merged'
count = 0
for entry in raw_datasets['train']:
    if entry['label'] == 1:
        count += 1

print(count)
print(len(raw_datasets['train']) - count)

574
574


In [20]:
# count how many entries in the test set are 'merged' and 'not_merged'
count = 0
for entry in raw_datasets['validation']:
    if entry['label'] == 1:
        count += 1

print(count)
print(len(raw_datasets['validation']) - count)

144
144


In [21]:
# tokenize the data
from transformers import AutoTokenizer

# Load the BERT tokenizer
checkpoint = 'bert-base-cased'
# checkpoint = 'google-bert/bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example['message'], padding="max_length", truncation=True)

# tokenize the training and validation sets
# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
print(tokenized_datasets.column_names)

/home/eric/uzh/pr-prediction/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/1148 [00:00<?, ? examples/s]

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

{'train': ['message', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'validation': ['message', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['message', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask']}


In [22]:
# from transformers import DataCollatorWithPadding

# data_collator = DataCollatorWithPadding(tokenizer)

In [23]:
import torch

torch.cuda.is_available()

True

In [24]:
print(torch.__version__)

2.3.0+cu118


In [25]:
import accelerate
print(accelerate.__version__)

0.30.1


In [26]:
from transformers import AutoModelForSequenceClassification, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

training_args = TrainingArguments("test-trainer")

/home/eric/uzh/pr-prediction/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
import numpy as np
import evaluate
from sklearn.metrics import roc_auc_score

accuracy_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')
precision_metric = evaluate.load('precision')
recall_metric = evaluate.load('recall')
# metric = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate each metric
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='macro')
    precision = precision_metric.compute(predictions=predictions, references=labels, average='macro', zero_division=0)
    recall = recall_metric.compute(predictions=predictions, references=labels, average='macro')
 
    # For AUC-ROC, we need to calculate it using sklearn if it's a binary classification
    if logits.shape[1] == 2:  # binary classification
        # Compute probabilities as softmax of logits
        probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
        # Select probability of positive class
        prob_positive = probabilities[:, 1]
        auc_roc = roc_auc_score(labels, prob_positive)
    else:
        auc_roc = None  # AUC-ROC is not applicable for non-binary classification directly

    result = {
        "accuracy": accuracy['accuracy'],
        "f1": f1['f1'],
        "precision": precision['precision'],
        "recall": recall['recall'],
    }

  # Add AUC-ROC if it's computed
    if auc_roc is not None:
        result["auc_roc"] = auc_roc

    return result

    # return metric.compute(predictions=predictions, references=labels)

from torch import zero_
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [29]:
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=33).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(300))
small_train_dataset = tokenized_datasets["train"]
# small_eval_dataset = tokenized_datasets["test"]

print(len(small_train_dataset))
print(len(small_eval_dataset))

1148
300


In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Auc Roc
1,No log,0.692904,0.503333,0.334812,0.252508,0.496711,0.573035
2,No log,0.675079,0.590000,0.503866,0.758038,0.584548,0.590283
3,No log,0.609401,0.623333,0.600984,0.665861,0.626600,0.681943


TrainOutput(global_step=432, training_loss=0.6753804242169416, metrics={'train_runtime': 96.3299, 'train_samples_per_second': 35.752, 'train_steps_per_second': 4.485, 'total_flos': 906154474659840.0, 'train_loss': 0.6753804242169416, 'epoch': 3.0})

In [32]:
evaluation_results = trainer.evaluate(eval_dataset=tokenized_datasets['validation'])
print(evaluation_results)

{'eval_loss': 0.5940009951591492, 'eval_accuracy': 0.6597222222222222, 'eval_f1': 0.6491472328576401, 'eval_precision': 0.6816187760473789, 'eval_recall': 0.6597222222222222, 'eval_auc_roc': 0.7260320216049383, 'eval_runtime': 2.4877, 'eval_samples_per_second': 115.767, 'eval_steps_per_second': 14.471, 'epoch': 3.0}
